# ADALINE

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np

# 生成男性数据
male = {
    "height": np.random.normal(171, 6, 500),  # 身高
    "weight": np.random.normal(70, 10, 500),  # 体重
    "bfr": np.random.normal(16, 2, 500),  # 体脂率
    "label": [1] * 500  # 标签
}

# 生成女性数据
female = {
    "height": np.random.normal(158, 5, 500),
    "weight": np.random.normal(57, 8, 500),
    "bfr": np.random.normal(22, 2, 500),
    "label": [-1] * 500
}

# 训练数据
train_data = np.array([
    np.concatenate((male["height"], female["height"])),
    np.concatenate((male["weight"], female["weight"])),
    np.concatenate((male["bfr"], female["bfr"])),
    np.concatenate((male["label"], female["label"]))
]).T
np.random.shuffle(train_data)  # 打乱数据

print(train_data.shape)
print(train_data)

(1000, 4)
[[152.47560124  59.93936362  17.29624965  -1.        ]
 [161.15564709  66.24509042  21.737371    -1.        ]
 [161.09316657  50.32841232  19.40931152  -1.        ]
 ...
 [168.67063194  66.21870382  15.61681455   1.        ]
 [159.35716567  55.03147636  25.22638551  -1.        ]
 [151.51401658  57.72611297  20.31658169  -1.        ]]


In [3]:
# 训练
from mflow import core, ops, opt

# 超参数
lr = 0.01
epoch = 50
batch_size = 16

with core.NameScope("ADALINE"):
    # 初始化变量
    x = core.Variable(size=(3, 1), trainable=False)
    y = core.Variable(size=(1, 1), trainable=False)
    w = core.Variable(size=(1, 3), trainable=True)
    b = core.Variable(size=(1, 1), trainable=True)
    # 模型定义
    pred = ops.Add(ops.MatMal(w, x), b)
    predicter = ops.Step(pred)
    loss = ops.loss.PerceptionLoss(ops.MatMal(y, pred))
    sgd = opt.AdaGrad(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, data in enumerate(train_data):
            x.setValue(np.mat(data[:-1]).T)
            y.setValue(np.mat(data[-1]))
            # # 前向
            # loss.forward()
            # # 反向
            # w.backward(loss)
            # b.backward(loss)
            # # 更新参数
            # w.step(lr)
            # b.step(lr)
            # # 清除图中的jacobi
            # core.DefaultGraph.clearAllJacobis()
            # 上面的代码用下面的优化器代替了
            sgd.step()
            bs_idx += 1
            if bs_idx == batch_size:
                sgd.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for data in train_data:
            x.setValue(np.mat(data[:-1]).T)
            predicter.forward()
            preds.append(predicter.value[0, 0])  # 结果
        preds = np.array(preds) * 2 - 1  # 0/1转为-1/1
        acc = (train_data[:, -1] == preds).astype("uint8").sum() / len(train_data)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.571.
Epoch: 2, acc: 0.946.
Epoch: 3, acc: 0.909.
Epoch: 4, acc: 0.961.
Epoch: 5, acc: 0.970.
Epoch: 6, acc: 0.973.
Epoch: 7, acc: 0.842.
Epoch: 8, acc: 0.877.
Epoch: 9, acc: 0.952.
Epoch: 10, acc: 0.921.
Epoch: 11, acc: 0.914.
Epoch: 12, acc: 0.974.
Epoch: 13, acc: 0.971.
Epoch: 14, acc: 0.832.
Epoch: 15, acc: 0.738.
Epoch: 16, acc: 0.933.
Epoch: 17, acc: 0.607.
Epoch: 18, acc: 0.826.
Epoch: 19, acc: 0.785.
Epoch: 20, acc: 0.826.
Epoch: 21, acc: 0.856.
Epoch: 22, acc: 0.800.
Epoch: 23, acc: 0.954.
Epoch: 24, acc: 0.820.
Epoch: 25, acc: 0.818.
Epoch: 26, acc: 0.834.
Epoch: 27, acc: 0.801.
Epoch: 28, acc: 0.896.
Epoch: 29, acc: 0.728.
Epoch: 30, acc: 0.784.
Epoch: 31, acc: 0.830.
Epoch: 32, acc: 0.970.
Epoch: 33, acc: 0.961.
Epoch: 34, acc: 0.696.
Epoch: 35, acc: 0.788.
Epoch: 36, acc: 0.943.
Epoch: 37, acc: 0.872.
Epoch: 38, acc: 0.785.
Epoch: 39, acc: 0.976.
Epoch: 40, acc: 0.889.
Epoch: 41, acc: 0.730.
Epoch: 42, acc: 0.769.
Epoch: 43, acc: 0.791.
Epoch: 44, acc: 0.79